In [1]:
from IPython.display import set_matplotlib_formats
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import logging
import jax
from math import log, exp, sqrt
from sim import sim_and_fit, sim_wf
from common import Observation
from plotting import plot_summary, compute_rmse, bias_variance

In [2]:
#Load sampling times
samples = pd.read_csv("aadr_1240k_EUR_4500BP_1kg_meta.txt", sep="\t")
samples["GenBP"] = samples.DateBP // 29  # assume 29 years per-generation
counts = samples.GenBP.value_counts().sort_index()
sizes, times = counts.values, np.array(counts.index)
T = times[-1]
mdl = {"s": [0.01] * T, "h": [0.5] * T, "f0": 0.1}

In [ ]:
#logging.basicConfig(level=logging.DEBUG)
#Runs ok
res1 = sim_and_fit(mdl, seed=123456, lam=1e2, Ne=10000, k=times, n=sizes)
print(res1['s'])

In [ ]:
#But if we increase lambda it just converges to zero
res2 = sim_and_fit(mdl, seed=12345, lam=1e3, Ne=10000, k=times, n=sizes)
print(res2['s'])